In [11]:
%%file formulation.lp

#const intlen = 32.

register(eax;ebx;ecx;edx;esi;edi;edp;esp;eip;imm;cmp).
flag(c;z;s;o).
apitype_value(get_something).
apitype_alloc(alloc_something).
pos(0..intlen-1).
time(0..maxtime).
pc(0..maxpc).

%% Generalizations
mathop(T, cmp) :- time(T), pcat(T, PC), cmp_imm(PC, _, _).
mathop(T, Reg) :- time(T), pcat(T, PC), add(PC, _, _, Reg).
mathop(T, Reg) :- time(T), pcat(T, PC), sub(PC, _, _, Reg).
imm(T) :- time(T), pcat(T, PC), cmp_imm(PC, _, _).
    
%% Jump detection
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jmp_abs(PC, Pos).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jo_abs(PC, Pos), flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jno_abs(PC, Pos), not flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), js_abs(PC, Pos), flg(T, s).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jns_abs(PC, Pos), not flg(T, s).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jz_abs(PC, Pos), flg(T, z).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jnz_abs(PC, Pos), not flg(T, z).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jc_abs(PC, Pos), flg(T, c).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jnc_abs(PC, Pos), not flg(T, c).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jl_abs(PC, Pos), flg(T, s), not flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jl_abs(PC, Pos), not flg(T, s), flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jge_abs(PC, Pos), flg(T, s), flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jge_abs(PC, Pos), not flg(T, s), not flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jle_abs(PC, Pos), flg(T, s), not flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jle_abs(PC, Pos), not flg(T, s), flg(T, o).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jle_abs(PC, Pos), flg(T, z).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jle_abs(PC, Pos), not flg(T, s), not flg(T, o), not flg(T, z).
jump_abs(T, Pos) :- time(T), pos(Pos), pcat(T, PC), jle_abs(PC, Pos), flg(T, s), flg(T, o), not flg(T, z).
%jump(T) :- time(T), pcat(T, PC), jmp_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jo_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jno_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), js_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jns_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jz_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jnz_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jc_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jnc_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jl_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jle_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jg_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jge_rel(PC, _).
%jump(T) :- time(T), pcat(T, PC), jmp_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jo_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jno_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), js_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jns_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jz_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jnz_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jc_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jnc_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jl_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jle_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jg_reg(PC, _).
%jump(T) :- time(T), pcat(T, PC), jge_reg(PC, _).
    
%% Registers maintain value unless acted upon
regbit(T, Reg, Pos) :-
    time(T),
    register(Reg),
    pos(Pos),
    regbit(T-1, Reg, Pos),
    pcat(T, PC),
    not end(PC),
    not sub(PC, _, _, Reg),
    Reg != eip,
    Reg != eax,
    Reg != imm,
    Reg != cmp.
regbit(T, eax, Pos) :-
    time(T),
    pos(Pos),
    regbit(T-1, eax, Pos),
    pcat(T, PC),
    not end(PC),
    not apicall(PC, _).
    
%% PC advances each time step
regbit(T+1, eip, 0) :-
    time(T),
    not regbit(T, eip, 0),
    not jump_abs(T, _).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    regbit(T, eip, Pos),
    regbit(T+1, eip, Pos-1),
    not jump_abs(T, _).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    regbit(T, eip, Pos),
    not regbit(T, eip, Pos-1),
    not jump_abs(T, _).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    not regbit(T, eip, Pos),
    not regbit(T+1, eip, Pos-1),
    regbit(T, eip, Pos-1),
    not jump_abs(T, _).

% Set PC on Jump
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    jump_abs(T, Pos).
    
% Comparisons    

%% Store value from cmp_imm in imm
regbit(T, imm, Pos) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    cmp_imm(PC, _, Pos).

%% Reg - imm -> cmp
cmp(PC, Reg, imm, cmp) :- cmp_imm(PC, Reg, _).
    
%% MOV
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    mov(PC, Reg, Res),
    register(Reg),
    regbit(T, Reg, Pos).

%% Subtraction
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    cmp(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    cmp(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    cmp(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    cmp(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    cmp(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    cmp(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    cmp(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
    
%% Addition
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    add(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    add(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    add(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
regbit(T+1, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    add(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    add(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    add(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    add(PC, Reg1, Reg2, _),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
    
%% API Calls: single value
0 { regbit(T, eax, Pos) } 1 :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    apicall(PC, Call),
    apitype_value(Call).
    
%% Flags
flg(T, c) :- 
    time(T),
    mathop(T, _),
    carry(T, intlen).
flg(T, c) :- 
    time(T),
    not mathop(T, _),
    flg(T-1, c).
flg(T, z) :-
    time(T),
    mathop(T, Reg),
    not regbit(T, Reg, _).
flg(T, z) :-
    time(T),
    not mathop(T, _),
    flg(T-1, z).
flg(T, s) :-
    time(T),
    mathop(T, Reg),
    regbit(T, Reg, intlen-1).
flg(T, s) :-
    time(T),
    not mathop(T, _),
    flg(T-1, s).
flg(T, o) :-
    time(T),
    pcat(T, PC),
    add(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    register(Res),
    regbit(T-1, Reg1, intlen-1),
    regbit(T-1, Reg2, intlen-1),
    not regbit(T, Res, intlen-1).
flg(T, o) :-
    time(T),
    pcat(T, PC),
    add(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    register(Res),
    not regbit(T-1, Reg1, intlen-1),
    not regbit(T-1, Reg2, intlen-1),
    regbit(T, Res, intlen-1).
flg(T, o) :-
    time(T),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    register(Res),
    regbit(T-1, Reg1, intlen-1),
    not regbit(T-1, Reg2, intlen-1),
    not regbit(T, Res, intlen-1).
flg(T, o) :-
    time(T),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    register(Res),
    not regbit(T-1, Reg1, intlen-1),
    regbit(T-1, Reg2, intlen-1),
    regbit(T, Res, intlen-1).
flg(T, o) :-
    time(T),
    not mathop(T, _),
    flg(T-1, o).


Overwriting formulation.lp


In [43]:
%%file find-malware.lp
    
malware_sig_eicar(T, PC) :- time(T), pcat(T, PC), eicar(PC).
    
malware_sig :-
    malware_sig_eicar(_, _).
    
:- not malware_sig.

Overwriting find-malware.lp


In [63]:
%%file instance-6.lp

#const maxtime = 20.
#const maxpc = 10.

apicall(1, get_something).
mov(2, eax, ebx).
apicall(3, get_something).
add(4, eax, ebx, eax).
cmp_imm(5, eax, 0).
cmp_imm(5, eax, 2).
cmp_imm(5, eax, 4).
cmp_imm(5, eax, 7).
cmp_imm(5, eax, 12).
cmp_imm(5, eax, 16).
cmp_imm(5, eax, 17).
cmp_imm(5, eax, 23).
jl_abs(6, 0).
jl_abs(6, 1).
jl_abs(6, 3).
cmp_imm(8, ebx, 3).
cmp_imm(8, ebx, 7).
jge_abs(9, 0).
jge_abs(9, 1).
jge_abs(9, 3).
eicar(10).
end(maxpc).

Overwriting instance-6.lp


In [ ]:
%%file instance-6.ins
.code
0x01 APICALL GetSomething
0x02 MOV EAX EBX
0x03 APICALL GetSomething
0x04 ADD EAX EBX EAX
0x05 CMP EAX 0x419095
0x06 JL 0xB
0x07 NOP
0x08 CMP EBX 0x88
0x09 JGE 0x01
0x0A EICAR
0x0B APICALL Exit

In [64]:
%%file suppress-warnings.lp

apicall(0, get_something) :- a=b.
cmp_imm(0, eax, zero) :- a=b.
jmp_abs(0, 0) :- a=b.
jo_abs(0, 0) :- a=b.
jno_abs(0, 0) :- a=b.
js_abs(0, 0) :- a=b.
jns_abs(0, 0) :- a=b.
jz_abs(0, 0) :- a=b.
jnz_abs(0, 0) :- a=b.
jc_abs(0, 0) :- a=b.
jnc_abs(0, 0) :- a=b.
jl_abs(0, 0) :- a=b.
jle_abs(0, 0) :- a=b.
jg_abs(0, 0) :- a=b.
jge_abs(0, 0) :- a=b.
jmp_rel(0, 0) :- a=b.
jo_rel(0, 0) :- a=b.
jno_rel(0, 0) :- a=b.
js_rel(0, 0) :- a=b.
jns_rel(0, 0) :- a=b.
jz_rel(0, 0) :- a=b.
jnz_rel(0, 0) :- a=b.
jc_rel(0, 0) :- a=b.
jnc_rel(0, 0) :- a=b.
jl_rel(0, 0) :- a=b.
jle_rel(0, 0) :- a=b.
jg_rel(0, 0) :- a=b.
jge_rel(0, 0) :- a=b.
jmp_reg(0, 0) :- a=b.
jo_reg(0, 0) :- a=b.
jno_reg(0, 0) :- a=b.
js_reg(0, 0) :- a=b.
jns_reg(0, 0) :- a=b.
jz_reg(0, 0) :- a=b.
jnz_reg(0, 0) :- a=b.
jc_reg(0, 0) :- a=b.
jnc_reg(0, 0) :- a=b.
jl_reg(0, 0) :- a=b.
jle_reg(0, 0) :- a=b.
jg_reg(0, 0) :- a=b.
jge_reg(0, 0) :- a=b.
eicar(0) :- a=b.
add(0, eax, eax, eax) :- a=b.
sub(0, eax, eax, eax) :- a=b.
mov(0, eax, eax) :- a=b.


Overwriting suppress-warnings.lp


In [65]:
%%file gen-pcat.py

import sys

bits = int(sys.argv[1])
maxpc = int(sys.argv[2])

with open('pcat' + str(bits) + '.lp', 'w') as f:
    f.write('toolong(T) :-\n')
    f.write('    time(T),\n')
    f.write('    regbit(T, eip, Pos),\n')
    f.write('    pos(Pos),\n')
    f.write('    Pos >= ' + str(maxpc.bit_length()) + '.\n\n')
    for i in range(0, maxpc + 1):
        f.write('pcat(T, ' + str(i) + ') :-\n')
        f.write('    time(T),\n')
        f.write('    pc(PC),\n')
        for j in range(0, maxpc.bit_length()):
            f.write('    ' + ('' if i&(1<<j) else 'not ') + 'regbit(T, eip, ' + str(j) + '),\n')
        f.write('    not toolong(T).\n')

Overwriting gen-pcat.py


In [66]:
%run gen-pcat 32 20

In [71]:
%%file diagnostics.lp

apicall_ret(T, PC) :-
    time(T),
    pcat(T, PC),
    apicall(PC, _).
    
apicall_res(T, Pos) :-
    apicall_ret(T, _),
    pos(Pos),
    regbit(T, eax, Pos).
    
#show apicall_ret/2.
#show apicall_res/2.
#show malware_sig_eicar/2.

Overwriting diagnostics.lp


In [72]:
!clingo formulation.lp instance-6.lp pcat32.lp find-malware.lp suppress-warnings.lp diagnostics.lp -c intlen=32 --outf=2 --time-limit=60 > trace.json

In [75]:
%%file visualizer.py

import json

maxtime = 16
maxpc = 8
maxtimelen = str(int(maxtime.bit_length()/4)+1)
maxpclen = str(int(maxpc.bit_length()/4)+1)

calls = {}
sigs = []

with open('trace.json') as f:
    data = json.loads(f.read())
    for fact in data["Call"][0]["Witnesses"][-1]["Value"]:
        ent = fact.split('(')
        ent[1] = ent[1][:-1].split(',')
        if ent[0] == "apicall_ret":
            if not int(ent[1][0]) in calls:
                calls[int(ent[1][0])] = {'PC' : int(ent[1][1]), 'value': 0}
            else:
                calls[int(ent[1][0])]['PC'] = int(ent[1][1])
        if ent[0] == "apicall_res":
            if not int(ent[1][0]) in calls:
                calls[int(ent[1][0])] = {'PC' : 0, 'value': 1<<int(ent[1][1])}
            else:
                calls[int(ent[1][0])]['value'] += 1<<int(ent[1][1])
        if ent[0][:12] == "malware_sig_":
            sigs.append({'type':ent[0][12:], 'time': int(ent[1][0]), 'PC': int(ent[1][1])})
            
for sig in sigs:
    print(("Signature {:s} encountered at time 0x{:0" + maxtimelen + "X} at PC 0x{:0" + maxpclen + "X}\n")\
          .format(sig['type'].upper(), sig['time'], sig['PC']))
    print(("{:" + str(int(maxtimelen)+2) + "s}\t{:" + str(int(maxpclen)+2) + "s}\t{:10s}").format("Time", "PC", "Value"))
    for call in sorted(calls.items()):
        print(("0x{:0" + maxtimelen + "X}\t0x{:0" + maxpclen + "X}\t0x{:08X}")\
             .format(call[0], call[1]['PC'], call[1]['value']))    

Writing visualizer.py
